In [1]:
import pandas as pd
import numpy as np

import onnx
import onnxruntime

from sklearn.preprocessing import LabelEncoder



def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


def train_label_encoder(data_file, selected_classes=None):
    df = pd.read_csv(data_file)
    if selected_classes is not None:
        df = df[df["Label"].isin(selected_classes)]
    labels = df["Label"].values
    label_encoder = LabelEncoder()
    label_encoder.fit(labels)
    return label_encoder

/tmp/ipykernel_379874/2796754810.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import pandas as pd

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define the experiment name and file path
NAME = "dataset"
PATH = "/home/adilson/Git/LERMO/data/dataset.csv"
PATH = "/home/adilson/Git/LERMO/experiments/a1-a2-a3-a4-actor5/train.csv"


# Load the training data
df_train = pd.read_csv(PATH)
df_train = df_train.groupby('Label').apply(lambda x: x.sample(n=500, replace=True, random_state=1)).reset_index(drop=True)
#X_train = df_train.drop(columns=["Label", "Actor"])
X_train = df_train.drop(columns=["Label"])
Y_train = df_train["Label"]

print(X_train.shape)
print(Y_train.shape)



df_test = pd.read_csv("/home/adilson/Git/LERMO/data/Luis_LR_dns.csv")
#df_test = pd.read_csv("/home/adilson/Git/LERMO/data/test.csv")
df_test = pd.DataFrame(df_test)
Y_TEST = df_test["Label"]
X_TEST = df_test.drop(columns=["Label"])
X_TEST = X_TEST.values.astype(np.float32)


print(X_TEST.shape)
print(Y_TEST.shape)

/tmp/ipykernel_379874/1556606270.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train = df_train.groupby('Label').apply(lambda x: x.sample(n=500, replace=True, random_state=1)).reset_index(drop=True)


(13500, 42)
(13500,)
(47776, 42)
(47776,)


In [8]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

models = [
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("SVM", SVC()),
    ("Random Forest", RandomForestClassifier()),
    ("Naive Bayes", GaussianNB())
]

column_names = ["Experiment Name", "Accuracy", "F1-score", "Recall", "Precision"]


results = []

for model_name, model in models:
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_TEST)
    accuracy = accuracy_score(Y_TEST, Y_pred)
    f1 = f1_score(Y_TEST, Y_pred, average='weighted')
    recall = recall_score(Y_TEST, Y_pred, average='weighted')
    precision = precision_score(Y_TEST, Y_pred, average='weighted')
    results.append([model_name, round(accuracy, 4) if not pd.isna(accuracy) else None,
                    round(f1, 4) if not pd.isna(f1) else None,
                    round(recall, 4) if not pd.isna(recall) else None,
                    round(precision, 4) if not pd.isna(precision) else None])


results_df = pd.DataFrame(results, columns=column_names)


pd.set_option("display.float_format", "{:.4f}".format)
print(results_df)

results_df.to_csv("experiments_sklearn.csv", index=False)

/home/adilson/miniconda3/envs/lermo/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/adilson/miniconda3/envs/lermo/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/home/adilson/miniconda3/envs/lermo/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/adilson/miniconda3/envs/lermo/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


       Experiment Name  Accuracy  F1-score  Recall  Precision
0  K-Nearest Neighbors    0.8495    0.8296  0.8495     0.8533
1                  SVM    0.8507    0.8283  0.8507     0.8559
2        Random Forest    0.8330    0.8023  0.8330     0.8447
3          Naive Bayes    0.7219    0.6902  0.7219     0.7368
